In [1]:
%pip install google-genai
%pip install matplotlib
%pip install os
%pip install dotenv
%pip install pydantic


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
ERROR: Could not find a version that satisfies the requirement os (from versions: none)

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
ERROR: No matching distribution found for os
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updat

In [2]:
from google import genai
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from dotenv import load_dotenv
from google.genai import types
from pydantic import BaseModel, Field

In [3]:
class Segment(BaseModel):
    start: str = Field(description="Start of a segment.")
    end: str = Field(description="End of a segment.")
    reason: str | None = Field(None, description="Reason why this segment is important.")

class GroundedVideoAnswer(BaseModel):
    answer: str = Field(..., description="answer")
    segments: list[Segment] = Field(..., description="a list of video segments")

In [4]:
def create_genai_client():
    import os

    api_key = os.getenv("GOOGLE_API_KEY")
    if not api_key:
        raise ValueError("GOOGLE_API_KEY not found in .env file")    

    # Configure the client with your API key
    client = genai.Client(api_key=api_key, http_options={'api_version': 'v1alpha'})

    return client


def create_vertexai_client():
    import os
    
    cloud_api_key = os.getenv("GOOGLE_CLOUD_API_KEY")
    if not cloud_api_key:
        raise ValueError("GOOGLE_CLOUD_API_KEY not found in .env file")
    
    # Configure the client with your API key
    client = genai.Client(
        vertexai=True, 
        api_key=cloud_api_key, 
    )

    return client

In [5]:
load_dotenv()

# Configure the client with your API key
client = create_vertexai_client()

In [15]:
def ask_my_youtube(file_uri: str, question: str):
    def clean_json_string(raw_string):
        # Remove the markdown code blocks
        clean_str = raw_string.strip()
        if clean_str.startswith("```json"):
            clean_str = clean_str[7:]
        if clean_str.endswith("```"):
            clean_str = clean_str[:-3]
        return clean_str.strip()
    
    response = client.models.generate_content(
        model='gemini-3-flash-preview',
        contents=types.Content(
            role="user",
            parts=[
                types.Part(
                    file_data=types.FileData(file_uri=file_uri, mime_type="video/mp4"),
                    video_metadata=types.VideoMetadata(fps=2)  # custom frame
                ),
                types.Part(text=question)
            ]
        ),
        config=types.GenerateContentConfig(
            response_mime_type="application/json",
            response_json_schema=GroundedVideoAnswer.model_json_schema(),
            media_resolution=types.MediaResolution.MEDIA_RESOLUTION_LOW,
            tools=[types.Tool(
                google_search=types.GoogleSearch()
            )],
        )
    )
    
    result = GroundedVideoAnswer.model_validate_json(clean_json_string(response.text))
    return result

def print_result(file_uri: str, question: str):
    result = ask_my_youtube(file_uri=file_uri, question=question)

    print("Question: ", question)
    print("Answer: ", result.answer)
    print ("Similarities:")
    for s in result.segments:
        print("- ", f"[{s.start} - {s.end}], reason: {s.reason}")

In [ ]:
file_uri = "https://youtu.be/v6B44n1V9no?si=lupM2r3kDGxFzzHr"
print_result(file_uri=file_uri, question="Summarize the video in three sentences")

In [ ]:
file_uri = "https://youtu.be/R9LZrysSil0?si=9GdrgaTwtIZfCas-"
print_result(file_uri=file_uri, question="What are the use cases shown in the video?")
